Rename a bunch of files using Git and renamer?



In [3]:
var execSync = require('child_process').execSync;
try {
    require.resolve('renamer');
} catch (e) {
    execSync('npm install renamer');
}
var renamer = require('renamer');
var renameUsingGit = (root, match, find, replace) => {
    var files = renamer.expand(path.join(root, match));
    var results = renamer.replace({
        files: files.filesAndDirs, 
        find: find, 
        replace: replace});
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git ' 
                 + '--work-tree=' + JSON.stringify(root)
                 + ' --git-dir=' + JSON.stringify(path.join(root, '.git'))
                 + ' mv ' + JSON.stringify(r.before) 
                 + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
(renameUsingGit);


[Function: renameUsingGit]

In [4]:
// test it
// rename all scss files to less
renameUsingGit(
    '/Users/briancullinan/Documents/portal',
    'src/**/*.scss',
    '.scss',
    '.less');

[ '/Users/briancullinan/Documents/portal/src/app/app.component.less',
  '/Users/briancullinan/Documents/portal/src/app/app.less',
  '/Users/briancullinan/Documents/portal/src/app/auth/login/login.component.less',
  '/Users/briancullinan/Documents/portal/src/app/auth/register-form/register-form.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/footer/footer.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/layout.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/menu/menu.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/not-found/not-found.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/toolbar/toolbar.component.less',
  '/Users/briancullinan/Documents/portal/src/app/temp-demo-pages/contact/contact.component.less',
  '/Users/briancullinan/Documents/portal/src/app/temp-demo-pages/demo/demo.component.less',
  '/Users/briancullinan/Documents/portal/src/app/temp-demo-pages/h

How to use memory-fs and rewire to audit cli events?



In [1]:
var path = require('path');
var execSync = require('child_process').execSync;
try {
    require.resolve('@angular/cli');
    require.resolve('mock-require');
    require.resolve('typescript')
    require.resolve('memory-fs');
    require.resolve('rewire');
} catch (e) {
    execSync('npm install @angular/cli mock-require rewire typescript memory-fs');
}
var mock = require('mock-require');

var mountAndRewireAngular = (memfs) => {
    var fs = require('fs');
    var fsMock = {};
    Object.assign(fsMock, fs);
    var prototype = Object.getPrototypeOf(memfs);
    Object.keys(prototype)
        .forEach(k => {
            var orig = prototype[k].bind(memfs);
            fsMock[k] = (a1, a2, a3, a4, a5) => {
                try {
                    return orig(a1, a2, a3, a4, a5);
                } catch (e) {
                    return fs[k](a1, a2, a3, a4, a5);
                }
            }
        });
    
    ['existsSync']
        .forEach(k => {
            var orig = fsMock[k];
            fsMock[k] = (a1, a2, a3, a4) => {
                return orig(a1, a2, a3, a4)
                    || fs[k](a1, a2, a3, a4);
            }
        });
    
    ["stat", "readdir", "mkdirp", "rmdir", "unlink", "readlink"]
        .forEach(k => {
            var orig = fsMock[k];
            fsMock[k] = (path, cb) => {
                orig(path, (err, result) => {
                    if(err) {
                        return fs[k](path, cb);
                    }
                    if(typeof cb == 'function') {
                        return cb(err, result);
                    }
                })
            };
        });
    
    ["exists"]
        .forEach(k => {
            var orig = fsMock[k];
            fsMock[k] = (path, cb) => {
                orig(path, (result) => {
                    if(!result) {
                        return fs[k](path, cb);
                    }
                    if(typeof cb == 'function') {
                        return cb(result);
                    }
                });
            };
        });
    
    ["mkdir", "readFile"]
        .forEach(k => {
            var orig = fsMock[k];
            fsMock[k] = (path, opts, cb) => {
                orig(path, opts, (err, result) => {
                    if(err && err.code != 'EEXIST') {
                        return fs[k](path, opts, cb);
                    }
                    if(typeof opts == 'function') {
                        return opts(err, result);
                    } else if(typeof cb == 'function') {
                        return cb(err, result);
                    }
                });
            };
        });
    
    mock('fs', fsMock);

    var ts = require('typescript');
    var tsMock = {};
    Object.assign(tsMock, ts);
    var toMock = {
        readFile: 'readFileSync',
        //writeFile: 'writeFileSync',
        //fileExists: 'existsSync',
        //directoryExists: 'existsSync',
        //createDirectory: 'mkdirSync',
        //getDirectories: 'readdirSync'
    };
    Object.keys(ts.sys)
        .forEach(k => {
            var orig = tsMock.sys[k];
            tsMock.sys[k] = (a1, a2, a3, a4, a5) => {
                if(typeof toMock[k] !== 'undefined') {
                    try {
                        return fsMock[toMock[k]](a1, a2, a3, a4, a5);
                    } catch (e) {
                        return;
                    }
                }
                return orig(a1, a2, a3, a4, a5);
            };
        });
    mock('typescript', tsMock);
    
    return require('@angular/cli');
};
(mountAndRewireAngular);



SyntaxError: Unexpected token .